## Requirements:

- Download the [MedProc Test Data](https://temu.bsc.es/medprocner/downloads/) into the folder `medprocner_train_test`
- Aquier an OpenAI API key

## Subtask 1

In [ ]:
%pip install openai

In [ ]:
import os
import csv
import time
import openai
import json

openai_key = input()
openai.api_key = openai_key
model_name = "gpt-3.5-turbo"
#model_name = "gpt-4"

def extract_procedures(text, examples):
    # Convert the examples into a conversation format
    conversation = [{'role': 'system', 'content': """Eres un asistente útil que extrae procedimientos médicos de textos médicos en español. 
    Un procedimiento médico se refiere a cualquier acción diagnóstica, terapéutica, médica o quirúrgica realizada en un paciente. 
    Tu respuesta debe ser una lista de procedimientos en formato JSON válido."""}]
    for input, output in examples:
        conversation.append({'role': 'user', 'content': f'{input}'})
        conversation.append({'role': 'assistant', 'content': json.dumps(output)})
    conversation.append({'role': 'user', 'content': f"""Extraiga todos los procedimientos médicos 
    del texto delimitado por tres comillas invertidas. Devuelve una lista vacía si no se menciona ninguno.
    ```{text}```"""})
    
    response = openai.ChatCompletion.create(
        model=model_name,
        messages=conversation
    )
    print("prompt:")
    print(conversation)
    print("response:")
    print(response)

    procedures = json.loads(response['choices'][0]['message']['content'])
    print(procedures)
    return procedures

# Call the function with a text and examples from the training set
examples = [("""
Extraiga todos los procedimientos médicos del texto delimitado por tres comillas invertidas. Devuelve una lista vacía si no se menciona ninguno.:
```Paciente varón de 35 años con tumoración en polo superior de teste derecho hallada de manera casual durante una autoexploración, motivo por el cual acude a consulta de urología donde se realiza exploración física, apreciando masa de 1cm aproximado de diámetro dependiente de epidídimo, y ecografía testicular, que se informa como lesión nodular sólida en cabeza de epidídimo derecho. Se realiza RMN. Confirmando masa nodular, siendo el tumor adenomatoide de epidídimo la primera posibilidad diagnóstica.

Se decide, en los dos casos, resección quirúrgica de tumoración nodular en cola epidídimo derecho, sin realización de orquiectomía posterior.
En ambos casos se realizó examen anátomopatológico de la pieza quirúrgica. Hallazgos histológicos macroscópicos: formación nodular de 1,5 cms (caso1) y 1,2 cms (caso 2) de consistencia firme, coloración blanquecina y bien delimitada. Microscópicamente se observa proliferación tumoral constituida por estructuras tubulares en las que la celularidad muestra núcleos redondeados y elongados sin atipia citológica y que ocasionalmente muestra citoplasmas vacuolados, todo ello compatible con tumor adenomatoide de epidídimo.
```""", ["examen anátomopatológico", "resección quirúrgica de tumoración nodular en cola epidídimo derecho", "orquiectomía", "exploración física", "ecografía testicular", "RMN", "Microscópicamente", "autoexploración", "histológicos macroscópicos"]), 
("""
Extraiga todos los procedimientos médicos del texto delimitado por tres comillas invertidas. Devuelve una lista vacía si no se menciona ninguno.:
```Dos recién nacidos, varón y hembra de una misma madre y fallecidos a los 10 y 45 minutos de vida respectivamente a los que se les realizó examen necrópsico. El primero de los cadáveres, correspondiente a la hembra, fue remitido con el juicio clínico de insuficiencia respiratoria grave con sospecha de Síndrome de Potter con la constatación de oligoamnios severo; nació mediante cesárea urgente por presentación de nalgas y el test de Apgar fue 1/3/7; minutos más tarde falleció. El examen externo permitió observar una tonalidad subcianótica, facies triangular con hendiduras parpebrales mongoloides, micrognatia, raiz nasal ancha y occipucio prominente. El abdomen, globuloso, duro y ligeramente abollonado permitía la palpación de dos grandes masas ocupando ambas fosas renales y hemiabdomenes. A la apertura de cavidades destacaba la presencia de dos grandes masas renales de 10 x 8 x 5,5 cm y 12 x 8 x 6 cm con pesos de 190 y 235 gr respectivamente. Si bien se podía discernir la silueta renal, la superficie, abollonada, presentaba numerosas formaciones quísticas de contenido seroso; al corte dichos quistes mostraban un tamaño heterogéneo siendo mayores los situados a nivel cortical, dando al riñón un aspecto de esponja. Los pulmones derecho e izquierdo pesaban 17 y 15 gr (peso habitual del conjunto de 49 gr) mostrando una tonalidad rojiza uniforme; ambos se encontraban comprimidos como consecuencia de la elevación diafragmática condicionada por el gran tamaño de los riñones. El resto de los órganos no mostraba alteraciones macroscópicas significativas salvo las alteraciones posicionales derivadas de la compresión renal. En el segundo de los cadáveres, el correspondiente al varón, se observaron cambios morfológicos similares si bien el tamaño exhibido por los riñones era aún mayor, con pesos de 300 y 310 gr. El resto de las vísceras abdominales estaban comprimidas contra el diafragma. En ambos casos se realizó un estudio histológico detallado, centrado especialmente en los riñones en los que se demostraron múltiples quistes de distintos tamaños con morfología sacular a nivel cortical. Dichos quistes ocupaban la mayor parte del parénquima corticomedular si bien las zonas conservadas no mostraban alteraciones significativas salvo inmadurez focal. Dichos quistes estaban tapizados por un epitelio simple que variaba desde plano o cúbico. Los quistes medulares, de menor tamaño y más redondeados estaban tapizados por un epitelio de predominio cúbico. Después de las renales, las alteraciones más llamativas se encontraban en el hígado donde se observaron proliferación y dilatación, incluso quística, de los ductos biliares a nivel de los espacios porta. Con tales hallazgos se emitió en ambos casos el diagnóstico de enfermedad poliquística renal autosómica recesiva infantil.
```""", 
[
"examen necrópsico",
"cesárea urgente",
"estudio histológico",
"examen externo",
"palpación",
"apertura de cavidades"]), ("""
Extraiga todos los procedimientos médicos del texto delimitado por tres comillas invertidas. Devuelve una lista vacía si no se menciona ninguno.:
```Paciente de 64 años, alérgico a penicilina y con recambio valvular aórtico por endocarditis que consultó por aparición de masa peneana de crecimiento progresivo en las últimas semanas. A la exploración física destacaba una formación excrecente y abigarrada en glande, que deformaba meato, con áreas ulceradas cubiertas de fibrina. Se palpaban adenopatías fijas y duras en ambas regiones inguinales. La radiografía de tórax y el TAC abdomino-pélvico confirmaron la presencia de adenopatías pulmonares e inguinales de gran tamaño. Con el diagnóstico de neoplasia de pene, se practicó penectomía parcial con margen de seguridad. La anatomía patológica demostró que se trataba de un sarcoma pleomórfico de pene con diferenciación osteosarcomatosa y márgenes libres de afectación. Se decidió tratamiento con dos líneas de quimioterapia consistente en adriamicina e ifosfamida pero no hubo respuesta. Ingresó de nuevo con recidiva local sangrante de gran tamaño y crecimiento rápido que provocaba obstrucción de meato con insuficiencia renal aguda. Se colocó sonda de cistostomía y se instauró tratamiento con sueroterapia, mejorando la función renal, pero con empeoramiento progresivo del estado general hasta que falleció a los 6 meses del diagnóstico.
```""", [
"recambio valvular aórtico", 
"exploración física", 
"Se palpaban adenopatías fijas y duras en ambas regiones inguinales", 
"radiografía de tórax", 
"TAC abdomino-pélvico", 
"penectomía parcial con margen de seguridad", 
"anatomía patológica", 
"quimioterapia",
"colocó sonda de cistostomía",
"sueroterapia"])] 

def process_files(directory):
    index = 0
    offset = 0
    for filename in os.listdir(directory):
        print(filename)
        index = index +1
        if filename.endswith('.txt') and index >offset:
            print(index)
            with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
                text = file.read()
                try:
                    procedures = extract_procedures(text, examples)
                except Exception as err:
                    print(err)
                    print("error while processing request, retry")
                    time.sleep(5)
                    procedures = extract_procedures(text, examples)
                with open(f'subtask1_result.tsv', 'a', newline='', encoding='utf-8') as tsvfile:
                    writer = csv.writer(tsvfile, delimiter='\t')
                    for procedure in procedures:
                        start_span = text.find(procedure)
                        end_span = start_span + len(procedure)
                        if start_span>=0 and end_span >= 0:
                            writer.writerow([filename.replace('.txt', ''), 'PROCEDIMIENTO', start_span, end_span, procedure])

# Call the function with the directory containing the text files
process_files('./medprocner_train_test/medprocner_train_test/medprocner_test/txt')


## Subtask 2

##### Install Dependencies

In [ ]:
%pip install pandas fuzzywuzzy python-Levenshtein nltk scipy
import nltk
nltk.download('punkt')

#### Define and Initialize Gazeteer Lookup

In [ ]:
import csv
from rapidfuzz import fuzz, process
from nltk.stem.snowball import SnowballStemmer


stemmer = SnowballStemmer("spanish") 


def initialize_gazetteer(filepath):
    with open(filepath, 'r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file, delimiter='\t')
        next(reader)  # Skip the header
        rows = [row for row in reader if row[2] == 'procedure']  # Filter only procedures
        terms = [' '.join(stemmer.stem(word) for word in row[1].split()) for row in rows]
        codes = [row[0] for row in rows]
    return terms, codes


def lookup(query, terms, codes, limit=1):
    stemmed_query = ' '.join(stemmer.stem(word) for word in query.split())
    result = process.extractOne(stemmed_query, terms, scorer=fuzz.ratio, score_cutoff=80)
    codes_dict = []
    if result:
        print(result)
        match, score, i = result
        if match in terms:
            index = terms.index(match)
            code = codes[index]
            codes_dict.append((match, code))
    return codes_dict

# Initialize the gazetteer
terms, codes = initialize_gazetteer('./medprocner_train_test/medprocner_train_test/medprocner_gazetteer/gazzeteer_medprocner_v1.tsv')

# Perform a test lookup
print(lookup('ecografía escrotal', terms, codes))




#### Define Processing Function for Test Data

In [66]:
import csv

def process_entries(input_filepath, terms, codes, output_filepath):
    with open(input_filepath, 'r', newline='', encoding='utf-8') as infile, open(output_filepath, 'w', newline='', encoding='utf-8') as outfile:
        reader = csv.reader(infile, delimiter='\t')
        writer = csv.writer(outfile, delimiter='\t')

        for row in reader:
            # Parse the row
            filename, label, start_span, end_span, text = row

            # Lookup SNOMED CT codes
            print("lookup:"+text)
            result_codes = lookup(text, terms, codes)
            print(result_codes)
            # Concatenate the codes with "+" if there are multiple codes
            code_str = "+".join(str(code) for _, code in result_codes)

            # Write the row to the output file
            writer.writerow([filename, label, start_span, end_span, text, code_str])



#### Run

In [ ]:
process_entries("./subtask1_result.tsv", terms, codes, "./subtask2_result.tsv")


## Subtask 3

In [71]:
import csv
from collections import defaultdict

def create_indexed_file(input_file, output_file):
    data = defaultdict(list)

    # Read TSV file
    with open(input_file, 'r', newline='', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter='\t')
        for row in reader:
            if len(row) == 6:  # To avoid empty or incomplete lines
                filename, _, _, _, _, code = row
                if code:  # Check if code is not empty
                    data[filename].append(code)

    # Write to new TSV file
    with open(output_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f, delimiter='\t')
        writer.writerow(['filename', 'code'])  # Write header
        for filename, codes in data.items():
            writer.writerow([filename, '+'.join(codes)])

# Run Index
create_indexed_file('substask2_result.tsv', 'subtask3_result.tsv')